## Importing the pdf file

In [3]:
import fitz 

def extract_text_from_pdf(pdf_path):

    doc = fitz.open(pdf_path)
    text = ""
    
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)  
        text += page.get_text()  
    return text


pdf_path = "JAVA_PROGRAMMING.pdf" 
pdf_text = extract_text_from_pdf(pdf_path)
print(pdf_text[:50])

print("Extraction complete")


DIGITAL NOTES 
ON 
JAVA PROGRAMMING (R20A0508) 
 

Extraction complete


In [ ]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from reportlab.lib import colors
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from reportlab.pdfgen import canvas

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Loading the fine-tuned model
model_path = "./flan-t5-base-finetuned"
model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_path)

BLOOM_LEVELS = {
    1: "Remember",
    2: "Understand",
    3: "Apply",
    4: "Analyze",
    5: "Evaluate",
    6: "Create"
}

def generate_question(context, bloom_levels, max_length=50):
    level_prompts = [f"{BLOOM_LEVELS[level]} level" for level in bloom_levels]
    level_prompt = " and ".join(level_prompts)
    prompt = f"Generate a {level_prompt} question based on this text: {context}"
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
    outputs = model.generate(input_ids, max_length=max_length, num_return_sequences=1, 
                             do_sample=True, temperature=0.7, top_k=50)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def get_bloom_levels():
    print("Select Bloom's Taxonomy level(s) for this question:")
    for key, value in BLOOM_LEVELS.items():
        print(f"{key}. {value}")
    levels = input("Enter the numbers of the desired levels (comma-separated): ")
    return [int(level.strip()) for level in levels.split(',')]

def generate_test(num_questions=5, context_length=20000):
    test_questions = []
    
    
    for i in range(num_questions):
        print(f"\nQuestion {i+1}:")
        bloom_levels = get_bloom_levels()
        
        start = torch.randint(0, max(1, len(pdf_text) - context_length), (1,)).item()
        context = pdf_text[start:start+context_length]
        
        question = generate_question(context, bloom_levels)
        test_questions.append((question, bloom_levels))
    
    return test_questions


from reportlab.lib import colors
from reportlab.lib.pagesizes import letter, inch
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer, Image
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import mm

def create_pdf(questions, subject, year, ia_number):
    doc = SimpleDocTemplate("question_paper.pdf", pagesize=letter, topMargin=0.5*inch, bottomMargin=0.5*inch, leftMargin=0.5*inch, rightMargin=0.5*inch)
    styles = getSampleStyleSheet()
    elements = []

   
    logo = Image('university_logo.png', width=50*mm, height=25*mm)
    elements.append(logo)
    elements.append(Spacer(1, 0.2*inch))

  
    info_style = ParagraphStyle(name='Info', fontSize=14, alignment=1, spaceAfter=10)
    info = Paragraph(f"{subject}<br/>Year: {year}<br/>IA {ia_number}", info_style)
    elements.append(info)
    elements.append(Spacer(1, 0.2*inch))

 
    data = [['SL.\nNo.', 'Question', 'Marks', "Bloom's\nLevel"]]
    for i, (question, levels) in enumerate(questions, 1):
        level_names = [BLOOM_LEVELS[level] for level in levels]
       
        wrapped_question = Paragraph(question, styles['Normal'])
        data.append([str(i), wrapped_question, '10', ', '.join(level_names)])

   
    table = Table(data, colWidths=[0.6*inch, 4.3*inch, 0.6*inch, 1.5*inch])
    table.setStyle(TableStyle([
        ('BACKGROUND', (0,0), (-1,0), colors.grey),
        ('TEXTCOLOR', (0,0), (-1,0), colors.whitesmoke),
        ('ALIGN', (0,0), (-1,-1), 'CENTER'),
        ('VALIGN', (0,0), (-1,-1), 'MIDDLE'),
        ('FONTNAME', (0,0), (-1,0), 'Helvetica-Bold'),
        ('FONTSIZE', (0,0), (-1,0), 12),
        ('BOTTOMPADDING', (0,0), (-1,0), 12),
        ('BACKGROUND', (0,1), (-1,-1), colors.beige),
        ('TEXTCOLOR', (0,1), (-1,-1), colors.black),
        ('ALIGN', (1,1), (1,-1), 'LEFT'), 
        ('FONTNAME', (0,1), (-1,-1), 'Helvetica'),
        ('FONTSIZE', (0,1), (-1,-1), 10),
        ('TOPPADDING', (0,1), (-1,-1), 12),  
        ('BOTTOMPADDING', (0,1), (-1,-1), 12), 
        ('GRID', (0,0), (-1,-1), 1, colors.black),
        ('WORDWRAP', (0,0), (-1,-1)),  
    ]))
    elements.append(table)

    doc.build(elements)


subject = input("Enter the subject name: ")
year = input("Enter the year: ")
ia_number = input("Enter the IA number: ")


test = generate_test(num_questions=5)


create_pdf(test, subject, year, ia_number)

print("\nPDF generated as 'question_paper.pdf'")


Question 1:
Select Bloom's Taxonomy level(s) for this question:
1. Remember
2. Understand
3. Apply
4. Analyze
5. Evaluate
6. Create


Token indices sequence length is longer than the specified maximum sequence length for this model (5287 > 512). Running this sequence through the model will result in indexing errors



Question 2:
Select Bloom's Taxonomy level(s) for this question:
1. Remember
2. Understand
3. Apply
4. Analyze
5. Evaluate
6. Create

Question 3:
Select Bloom's Taxonomy level(s) for this question:
1. Remember
2. Understand
3. Apply
4. Analyze
5. Evaluate
6. Create

Question 4:
Select Bloom's Taxonomy level(s) for this question:
1. Remember
2. Understand
3. Apply
4. Analyze
5. Evaluate
6. Create

Question 5:
Select Bloom's Taxonomy level(s) for this question:
1. Remember
2. Understand
3. Apply
4. Analyze
5. Evaluate
6. Create

PDF generated as 'question_paper.pdf'
